<a href="https://colab.research.google.com/github/RissanX/food-sales-predicitons/blob/main/Food_Sales_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import pandas as pd
filename= '/content/drive/MyDrive/DataSets/sales_predictions_2023.csv'

In [51]:
df= pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [52]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [53]:
df.shape

(8523, 12)

In [54]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [55]:
df.info

<bound method DataFrame.info of      Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT049 

In [56]:
df[df.duplicated(keep=False)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [57]:
df.duplicated().sum()

0

In [58]:
df.duplicated(['Item_Identifier']).sum()

6964

In [59]:
df.duplicated(['Item_Identifier']).value_counts()

True     6964
False    1559
dtype: int64

In [60]:
df.duplicated(['Item_Identifier'],keep=False)

0       True
1       True
2       True
3       True
4       True
        ... 
8518    True
8519    True
8520    True
8521    True
8522    True
Length: 8523, dtype: bool

In [61]:
df.drop_duplicates(['Item_Identifier'],inplace=True)
df.sample(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
925,DRL11,10.50,Low Fat,0.048116,Hard Drinks,159.0946,OUT045,2002,NaN,Tier 2,Supermarket Type1,4102.6596
1527,NCS42,8.60,Low Fat,0.069809,Household,90.5146,OUT017,2007,NaN,Tier 2,Supermarket Type1,729.7168
612,NCQ41,14.80,Low Fat,0.019464,Health and Hygiene,196.3794,OUT013,1987,High,Tier 3,Supermarket Type1,1950.7940
2738,FDV03,17.60,Low Fat,0.058043,Meat,154.6314,OUT013,1987,High,Tier 3,Supermarket Type1,1861.5768
3085,DRF51,15.75,Low Fat,0.165838,Dairy,36.6506,OUT046,1997,Small,Tier 1,Supermarket Type1,607.2096
217,FDP28,13.65,Regular,0.134976,Frozen Foods,260.0936,OUT010,1998,NaN,Tier 3,Grocery Store,260.9936
211,DRC27,13.80,Low Fat,0.058091,Dairy,245.1802,OUT035,2004,Small,Tier 2,Supermarket Type1,5650.6446
1314,FDV40,17.35,Low Fat,0.014689,Frozen Foods,74.5038,OUT035,2004,Small,Tier 2,Supermarket Type1,739.0380
531,NCM55,15.60,Low Fat,0.111686,Others,184.7924,OUT010,1998,NaN,Tier 3,Grocery Store,370.1848
2205,FDD56,15.20,Regular,0.104201,Fruits and Vegetables,176.5054,OUT018,2009,Medium,Tier 3,Supermarket Type2,2976.7918


In [62]:
df['Outlet_Size'].value_counts()

Medium    472
Small     445
High      186
Name: Outlet_Size, dtype: int64

In [63]:
df['Item_Fat_Content'].value_counts()

Low Fat    929
Regular    532
LF          61
reg         19
low fat     18
Name: Item_Fat_Content, dtype: int64

In [64]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('LF','Low Fat')

In [65]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('reg','Regular')

In [66]:
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('low fat','Low Fat')

In [67]:
df['Item_Fat_Content'].value_counts()

Low Fat    1008
Regular     551
Name: Item_Fat_Content, dtype: int64

In [68]:
df['Item_Type'].value_counts()

Fruits and Vegetables    220
Snack Foods              220
Household                170
Frozen Foods             155
Dairy                    125
Canned                   120
Baking Goods             119
Health and Hygiene        95
Soft Drinks               80
Meat                      80
Breads                    45
Hard Drinks               40
Starchy Foods             30
Others                    30
Breakfast                 20
Seafood                   10
Name: Item_Type, dtype: int64

In [69]:
df['Outlet_Type'].value_counts()

Supermarket Type1    1062
Grocery Store         183
Supermarket Type3     158
Supermarket Type2     156
Name: Outlet_Type, dtype: int64

In [70]:
print('Item outlet sales')
print('mean:',df['Item_Outlet_Sales'].mean(),'\n')
print('max:',df['Item_Outlet_Sales'].max(),'\n')
print('min:',df['Item_Outlet_Sales'].min(),'\n')

Item outlet sales
mean: 2192.905187427838 

max: 11445.102 

min: 37.2848 



In [71]:
print('Item MRP')
print('mean:',df['Item_MRP'].mean(),'\n')
print('max:',df['Item_MRP'].max(),'\n')
print('min:',df['Item_MRP'].min(),'\n')

Item MRP
mean: 140.9663558691469 

max: 265.2226 

min: 31.29 



In [72]:
print('Item Weight')
print('mean:',df['Item_Weight'].mean(),'\n')
print('max:',df['Item_Weight'].max(),'\n')
print('min:',df['Item_Weight'].min(),'\n')

Item Weight
mean: 12.804567049808428 

max: 21.35 

min: 4.555 

